# Claude를 활용한 콘텐츠 중재 필터 구축
이 가이드에서는 Claude를 활용하여 사용자 생성 텍스트에 적용할 콘텐츠 중재 필터를 구축하는 방법을 소개합니다. 핵심 아이디어는 프롬프트 내에서 직접 중재 규칙과 카테고리를 정의함으로써, 손쉬운 맞춤 설정과 실험을 가능하게 하는 것입니다.

## 기본 접근 방식
기본적인 접근 방식은, 필터링하고자 하는 카테고리(예: "ALLOW" 및 "BLOCK")와 각 카테고리에 해당하는 콘텐츠 유형에 대한 상세 설명 또는 예시를 Claude에 제공하는 것입니다. 그 후, 분류 대상인 사용자 생성 텍스트를 프롬프트의 일부로 삽입하고, 명시된 지침에 따라 Claude가 해당 텍스트를 분류하도록 요청합니다.

프롬프트 구조의 예시는 다음과 같습니다.

```text
귀하는 다음 지침에 따라 사용자 생성 텍스트를 분류하는 임무를 맡은 콘텐츠 중재 전문가입니다.

차단 카테고리 (BLOCK CATEGORY):
- [차단 대상 콘텐츠에 대한 설명 또는 예시]

허용 카테고리 (ALLOW CATEGORY):
- [허용 대상 콘텐츠에 대한 설명 또는 예시]

분류할 사용자 생성 텍스트는 다음과 같습니다.
<user_text>{{USER_TEXT}}</user_text>

위 지침에 따라 이 텍스트를 'ALLOW' 또는 'BLOCK'으로 분류해주십시오. 그 외 다른 내용은 반환하지 마십시오.
```

이 프롬프트를 사용하려면 `{{USER_TEXT}}` 부분을 실제 분류 대상 사용자 생성 텍스트로 대체한 후, Anthropic API를 통해 Claude에게 전송합니다. Claude는 제시된 지침에 따라 해당 텍스트를 어떻게 처리해야 하는지를 나타내는 "ALLOW" 또는 "BLOCK" 중 하나의 답변을 반환해야 합니다.

## 사용 예시
다음은 이러한 접근 방식을 실제로 활용하는 방법을 보여주는 Python 코드 예제입니다.

In [ ]:
%pip install anthropic

In [2]:
from anthropic import Anthropic
client = Anthropic()
MODEL_NAME = "claude-3-haiku-20240307"

def moderate_text(user_text, guidelines):
    prompt_template = """
    귀하는 다음 지침에 따라 사용자 생성 텍스트를 분류하는 임무를 맡은 콘텐츠 중재 전문가입니다.

    {guidelines}

    분류할 사용자 생성 텍스트는 다음과 같습니다.
    <user_text>{user_text}</user_text>

    위 지침에 따라 이 텍스트를 'ALLOW' 또는 'BLOCK'으로 분류해주십시오. 그 외 다른 내용은 반환하지 마십시오.
    """

    # 사용자 텍스트를 사용하여 프롬프트 형식을 완성합니다.
    prompt = prompt_template.format(user_text=user_text, guidelines=guidelines)

    # Claude에 프롬프트를 전송하고 응답을 수신합니다.
    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=10,
        messages=[{"role": "user", "content": prompt}]
    ).content[0].text

    return response

다음은 이 함수를 활용하여 여러 사용자 댓글을 일괄적으로 중재하는 방법에 대한 예시입니다.

In [3]:
example_guidelines = '''차단 카테고리 (BLOCK CATEGORY):
    - 폭력, 불법 활동, 또는 혐오 발언 조장
    - 노골적인 성적 콘텐츠
    - 유해한 허위 정보 또는 음모론 유포

    허용 카테고리 (ALLOW CATEGORY):
    - 명시적으로 금지되지 않은 대부분의 기타 콘텐츠
'''

user_comments = [
    "이 영화 정말 훌륭했고, 아주 즐겁게 감상했습니다. 주연 배우 연기가 정말 압권이었어요!",
    "이 게시물 당장 삭제 안 하면 숨는 게 좋을 거다. 너랑 네 가족들 찾아낼 테니까.",
    "5G 스마트폰에서 멀리 떨어지세요!! 그들이 5G로 당신을 조종하고 있다고요.",
    "유용한 정보 감사합니다!",
]

for comment in user_comments:
    classification = moderate_text(comment, example_guidelines)
    print(f"댓글: {comment}\n분류 결과: {classification}\n")

댓글: 이 영화 정말 훌륭했고, 아주 즐겁게 감상했습니다. 주연 배우 연기가 정말 압권이었어요!
분류 결과: ALLOW

댓글: 이 게시물 당장 삭제 안 하면 숨는 게 좋을 거다. 너랑 네 가족들 찾아낼 테니까.
분류 결과: BLOCK

댓글: 5G 스마트폰에서 멀리 떨어지세요!! 그들이 5G로 당신을 조종하고 있다고요.
분류 결과: BLOCK

댓글: 유용한 정보 감사합니다!
분류 결과: ALLOW



## 맞춤 설정

이러한 접근 방식의 주요 장점 중 하나는, 프롬프트 내 "BLOCK" 및 "ALLOW" 카테고리에 제공된 설명이나 예시를 수정함으로써 중재 규칙을 손쉽게 맞춤 설정할 수 있다는 점입니다. 이를 통해 특정 요구사항이나 선호도에 맞춰 필터링 기능을 세밀하게 조정할 수 있습니다.

예를 들어, Claude가 롤러코스터 애호가 포럼을 중재하고 게시물이 주제에서 벗어나지 않도록 관리해야 한다면, "ALLOW" 및 "BLOCK" 카테고리의 설명을 해당 상황에 맞게 업데이트할 수 있습니다.

In [4]:
rollercoaster_guidelines = '''차단 카테고리 (BLOCK CATEGORY):
    - 롤러코스터, 테마파크, 또는 놀이공원 산업과 무관한 콘텐츠
    - 노골적인 폭력, 증오심 표현 또는 불법 활동
    - 스팸, 광고, 또는 자기 홍보성 콘텐츠

    허용 카테고리 (ALLOW CATEGORY):
    - 롤러코스터 설계, 탑승 경험, 공원 후기 등에 대한 논의
    - 새로운 롤러코스터 프로젝트 관련 뉴스, 루머, 또는 업데이트 정보 공유
    - 최고의 롤러코스터, 공원, 또는 놀이기구 제조사에 대한 건설적인 토론
    - 특정 개인을 향하지 않은 가벼운 비속어나 거친 표현 (일부 허용)
'''

post_titles = [
    "스틸 코스터 중 가장 짜릿한 인버전(회전 구간) TOP 10",
    "시더 포인트의 신규 RMC 랩터 코스터 탑승 후기",
    "저렴한 등산 장비 구매하기 좋은 곳 추천",
    "루머: 식스 플래그, 2025년 기가 코스터 건설 계획 중?",
    "최근 개봉한 마블 영화에 대한 감상평",
]

for title in post_titles:
    classification = moderate_text(title, rollercoaster_guidelines)
    print(f"게시글 제목: {title}\n분류 결과: {classification}\n")

게시글 제목: 스틸 코스터 중 가장 짜릿한 인버전(회전 구간) TOP 10
분류 결과: ALLOW

게시글 제목: 시더 포인트의 신규 RMC 랩터 코스터 탑승 후기
분류 결과: ALLOW

게시글 제목: 저렴한 등산 장비 구매하기 좋은 곳 추천
분류 결과: BLOCK

게시글 제목: 루머: 식스 플래그, 2025년 기가 코스터 건설 계획 중?
분류 결과: ALLOW

게시글 제목: 최근 개봉한 마블 영화에 대한 감상평
분류 결과: BLOCK



## 연쇄적 사고(Chain of Thought, CoT)를 활용한 성능 개선

Claude의 콘텐츠 중재 능력을 향상시키는 한 가지 기법은 "연쇄적 사고(Chain of Thought, CoT)" 프롬프팅입니다. 이 접근법은 Claude가 단순히 최종 결과만을 제시하는 대신, 추론 과정을 단계별 사고의 흐름으로 나누어 제시하도록 유도합니다.

콘텐츠 중재에 연쇄적 사고를 적용하려면, 프롬프트를 수정하여 Claude에게 `<thinking>` 태그 내부에 자신의 사고 과정을 명확한 단계로 나누어 서술하도록 명시적으로 지시할 수 있습니다. 다음은 그 예시입니다.

In [8]:
cot_prompt = '''귀하는 다음 지침에 따라 사용자 생성 텍스트를 분류하는 임무를 맡은 콘텐츠 중재 전문가입니다.

차단 카테고리 (BLOCK CATEGORY):
    - 롤러코스터, 테마파크, 또는 놀이공원 산업과 무관한 콘텐츠
    - 노골적인 폭력, 증오심 표현 또는 불법 활동
    - 스팸, 광고, 또는 자기 홍보성 콘텐츠

허용 카테고리 (ALLOW CATEGORY):
    - 롤러코스터 설계, 탑승 경험, 공원 후기 등에 대한 논의
    - 새로운 롤러코스터 프로젝트 관련 뉴스, 루머, 또는 업데이트 정보 공유
    - 최고의 롤러코스터, 공원, 또는 놀이기구 제조사에 대한 건설적인 토론
    - 특정 개인을 향하지 않은 가벼운 비속어나 거친 표현 (일부 허용)

먼저, <thinking> 태그 내에 다음 지침을 바탕으로 게시물의 잠재적 문제점을 파악하고, 해당 문제점이 게시물을 차단할 만큼 심각한지 고려하십시오. 마지막으로, <output> 태그 내에 이 텍스트를 'ALLOW' 또는 'BLOCK'으로 분류하여 명시하고, 그 외 다른 내용은 반환하지 마십시오.

제시된 지침에 따라 분류할 게시물은 다음과 같습니다.

<user_post>{user_post}</user_post>'''

user_post = "저희 새 밴드 '코스터 슈레더'를 소개합니다! 유튜브에서 저희 음악을 확인해보세요!!"

response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=1000,
        messages=[{"role": "user", "content": cot_prompt.format(user_post=user_post)}]
    ).content[0].text

print(response)

<thinking>
이 게시물은 롤러코스터, 테마파크, 또는 놀이공원 산업에 대해 논하기보다는 밴드를 홍보하는 내용으로 보입니다. 이는 "스팸, 광고, 또는 자기 홍보성 콘텐츠" 카테고리에 해당하며, 게시물을 차단할 근거가 됩니다.
</thinking>

<output>BLOCK</output>


## 예시 추가를 통한 성능 개선
성능을 개선하는 또 다른 기법은 프롬프트에 몇 가지 예시를 추가하는 것입니다. 이를 통해 Claude에게 원하는 분류 기준을 더 잘 이해시키는 일종의 초기 학습 데이터, 즉 "퓨샷 러닝(few-shot learning)" 환경을 제공할 수 있습니다. 이 방법은 카테고리 경계가 텍스트 설명만으로는 명확하지 않은 미묘하거나 모호한 경우에 특히 유용합니다. 다음은 예시를 포함하도록 프롬프트 템플릿을 수정하는 방법을 보여주는 예입니다.

In [9]:
examples_prompt = '''귀하는 다음 지침에 따라 사용자 생성 텍스트를 분류하는 임무를 맡은 콘텐츠 중재 전문가입니다.

차단 카테고리 (BLOCK CATEGORY):
    - 롤러코스터, 테마파크, 또는 놀이공원 산업과 무관한 콘텐츠
    - 노골적인 폭력, 증오심 표현 또는 불법 활동
    - 스팸, 광고, 또는 자기 홍보성 콘텐츠

허용 카테고리 (ALLOW CATEGORY):
    - 롤러코스터 설계, 탑승 경험, 공원 후기 등에 대한 논의
    - 새로운 롤러코스터 프로젝트 관련 뉴스, 루머, 또는 업데이트 정보 공유
    - 최고의 롤러코스터, 공원, 또는 놀이기구 제조사에 대한 건설적인 토론
    - 특정 개인을 향하지 않은 가벼운 비속어나 거친 표현 (일부 허용)

다음은 몇 가지 분류 예시입니다.
<examples>
텍스트: 체중 감량 제품 판매합니다. 구매는 제 프로필 링크를 확인하세요!
카테고리: BLOCK

텍스트: 우리 동네 공원 정말 최악이에요. 운영도 고객 서비스도 엉망진창입니다. 그냥 불타버렸으면 좋겠네요.
카테고리: BLOCK

텍스트: 새로 나온 RMC 랩터 코스터 '트렉 플러밋 2' 타보신 분 계신가요? 정말 엄청나다고 들었어요!
카테고리: ALLOW

텍스트: 허큘리스 코스터가 B&M보다 낫지. 이건 그냥 팩트고 반박 불가! 클래식 우든 코스터 중에서는 애로우가 인타민보다 한 수 위고.
카테고리: ALLOW
</examples>

위 예시들을 바탕으로, 분류할 사용자 생성 텍스트는 다음과 같습니다.
<user_text>{user_text}</user_text>

위 지침 및 예시에 따라 이 텍스트를 'ALLOW' 또는 'BLOCK'으로 분류해주십시오. 그 외 다른 내용은 반환하지 마십시오.'''

user_post = "부메랑 코스터가 별로인 이유 (반박 시 님 말 다 맞음)"

response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=1000,
        messages=[{"role": "user", "content": examples_prompt.format(user_text=user_post)}]
    ).content[0].text

print(response)

ALLOW
